In [1]:
!pip install html2text
import requests
import html2text

def get_wikipedia_page_content(page_title):
    # API endpoint URL
    url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&explaintext=True&titles={page_title}"

    # Send GET request to the API endpoint
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Extract the page content from the response
        pages = data["query"]["pages"]
        page_content = list(pages.values())[0]["extract"]

        return page_content
    else:
        # Request failed
        print(f"Failed to retrieve Wikipedia page: {response.status_code}")

# Example usage
Dog = "Dog"  # Replace with the actual Wikipedia page title
Cat = "Cat"  # Replace with the actual Wikipedia page title
Pig = "Pig"  # Replace with the actual Wikipedia page title

page1_content = get_wikipedia_page_content(Dog)
page2_content = get_wikipedia_page_content(Cat)
test_page_content = get_wikipedia_page_content(Pig)

# Print the page content
print("Page 1 Content:")
print(page1_content)
print()

print("Page 2 Content:")
print(page2_content)
print()

print("Test Page Content:")
print(test_page_content)


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Page 1 Content:
The dog (Canis familiaris or Canis lupus familiaris) is a domesticated descendant of the wolf. Also called the domestic dog, it is derived from extinct Pleistocene wolves, and the modern wolf is the dog's nearest living relative. Dogs were the first species to be domesticated by hunter-gatherers over 15,000 years ago before the development of agriculture. Due to their long association with humans, dogs have expanded to a large number of domestic individuals and gained the ability to thrive on a starch-rich diet that would be inadequate for other canids.The dog has been selectively bred over millennia for various behaviors, sensory capabilities, and physical attributes. Dog breeds vary widely in shape, size, and color. They perform many roles for humans, such as hunting, herding, pulling loads, protection, assisting police and the military, companionship, therapy, and aiding disabled people. Over the millennia, dogs became uniquely adapted to human behavior, and the huma

In [28]:
!pip install nltk


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
import requests
from bs4 import BeautifulSoup

def get_wikipedia_page(url):
    # İstek göndererek sayfanın kaynak kodunu alalım
    response = requests.get(url)
    
    # HTML içeriğini BeautifulSoup kütüphanesi ile ayrıştıralım
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Sayfa içeriğini bulmak için uygun etiketleri ve sınıfları belirleyelim
    content_div = soup.find('div', {'id': 'mw-content-text'})
    paragraphs = content_div.find_all('p')
    
    # Paragrafları birleştirerek tam sayfa içeriğini oluşturalım
    page_content = '\n'.join([p.get_text() for p in paragraphs])
    
    return page_content

# İndirilecek Wikipedia sayfasının URL'sini belirtin
url = 'https://en.wikipedia.org/wiki/Cat'

# Wikipedia sayfasının içeriğini alalım
page_content = get_wikipedia_page(url)

# Dosya yolunu belirtin
file_path = 'C:\\Users\\ey\\Downloads\\cat.txt'

# Dosyayı oluşturup sayfa içeriğini yazalım
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(page_content)

print("Dosya başarıyla indirildi:", file_path)


Dosya başarıyla indirildi: C:\Users\ey\Downloads\cat.txt


In [79]:
import csv
import nltk
from nltk.corpus import sentiwordnet as swn

def extract_features(word, word_index):
    pos_tag = nltk.pos_tag([word])[0][1]
    previous_word = ""
    previous_pos_tag = ""
    if word_index > 0:
        previous_word = words[word_index - 1]
        previous_pos_tag = nltk.pos_tag([previous_word])[0][1]

    previous_two_words = ""
    previous_two_pos_tags = ["", ""]
    if word_index > 1:
        previous_two_words = words[word_index - 2]
        previous_two_pos_tags = [nltk.pos_tag([previous_two_words])[0][1], previous_pos_tag]

    next_word = ""
    next_pos_tag = ""
    if word_index < len(words) - 1:
        next_word = words[word_index + 1]
        next_pos_tag = nltk.pos_tag([next_word])[0][1]

    next_two_words = ""
    next_two_pos_tags = ["", ""]
    if word_index < len(words) - 2:
        next_two_words = words[word_index + 2]
        next_two_pos_tags = [nltk.pos_tag([next_two_words])[0][1], next_pos_tag]

    is_positive = False
    is_negative = False
    synsets = list(swn.senti_synsets(word))
    if synsets:
        for synset in synsets:
            if synset.pos_score() > synset.neg_score():
                is_positive = True
            elif synset.neg_score() > synset.pos_score():
                is_negative = True

    return [word, pos_tag, previous_pos_tag, previous_two_pos_tags[0], next_pos_tag, next_two_pos_tags[0], is_positive, is_negative]

input_file = 'Cat.txt'
output_file = 'ozellikler2.csv'

with open(input_file, 'r', encoding='utf-8') as file:
    text = file.read()
    words = text.split()

    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Word', 'POS', 'POS Before 1', 'POS Before 2', 'POS After 1', 'POS After 2', 'Positive', 'Negative'])

        for word_index, word in enumerate(words):
            features = extract_features(word, word_index)
            writer.writerow(features)

print("CSV dosyası başarıyla oluşturuldu.")

CSV dosyası başarıyla oluşturuldu.


In [82]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

# Veri kümesini yükleyin
data = pd.read_csv('ozellikler.csv')

# Giriş özelliklerini ve hedef değişkeni ayırın
X = data.drop(['Word', 'POS', 'Positive', 'Negative'], axis=1)  # Giriş özelliklerini seçin
y = data['Word']  # Hedef değişkeni seçin

# Kategorik özellikleri kodlayın
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_encoded = encoder.fit_transform(X)

# Eğitim ve test veri kümelerini ayırın
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Lojistik regresyon modelini oluşturun ve eğitin
model = LogisticRegression()
model.fit(X_train, y_train)

# Test veri kümesi üzerinde tahmin yapın
y_pred = model.predict(X_test)

# Modelin performansını değerlendirin
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)


C:\Users\ey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Model Accuracy: 0.1985535831689678


C:\Users\ey\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [85]:
import csv

# Somut kelimelerin olduğu dosyayı oku ve kelimeleri bir küme olarak yükle
concrete_words = set()
with open('400-700.txt', 'r') as file:
    reader = csv.reader(file, delimiter=' ')
    for row in reader:
        word = row[0]
        concrete_words.add(word)

# Soyut kelimelerin olduğu dosyayı oku ve kelimeleri bir küme olarak yükle
abstract_words = set()
with open('100-400.txt', 'r') as file:
    reader = csv.reader(file, delimiter=' ')
    for row in reader:
        word = row[0]
        abstract_words.add(word)

# Kendi oluşturduğunuz CSV dosyasını oku ve her bir kelime için soyut/somut bilgisini kontrol et
with open('ozellikler2.csv', 'r') as file:
    reader = csv.reader(file)
    
    # Yeni CSV dosyasını yazmak için çıktı dosyasını aç
    with open('sonuclar.csv', 'w', newline='') as output_file:
        writer = csv.writer(output_file)
        header = next(reader)  # Başlık satırını geç
        header.append('Abstract/Concrete')  # Yeni sütun başlığını ekle
        writer.writerow(header)  # Başlık satırını yaz

        # Her bir kelime için işlem yap
        for row in reader:
            word = row[0]  # Kelimeyi al

            # Kelimenin somut veya soyut olup olmadığını kontrol et ve ilgili değeri ekleyin
            if word in concrete_words:
                row.append('Somut')
            elif word in abstract_words:
                row.append('Soyut')
            else:
                row.append('')  # Eşleşme yoksa boş bırakın

            writer.writerow(row)  # Satırı yaz

print("Yeni CSV dosyası oluşturuldu.")


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 7184: character maps to <undefined>

In [86]:

import csv

# Somut kelimelerin olduğu dosyayı oku ve kelimeleri bir küme olarak yükle
concrete_words = set()
with open('400-700.txt', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter=' ')
    for row in reader:
        word = row[0]
        concrete_words.add(word)

# Soyut kelimelerin olduğu dosyayı oku ve kelimeleri bir küme olarak yükle
abstract_words = set()
with open('100-400.txt', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter=' ')
    for row in reader:
        word = row[0]
        abstract_words.add(word)

# Kendi oluşturduğunuz CSV dosyasını oku ve her bir kelime için soyut/somut bilgisini kontrol et
with open('ozellikler2.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    
    # Yeni CSV dosyasını yazmak için çıktı dosyasını aç
    with open('sonuclar.csv', 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        header = next(reader)  # Başlık satırını geç
        header.append('Abstract/Concrete')  # Yeni sütun başlığını ekle
        writer.writerow(header)  # Başlık satırını yaz

        # Her bir kelime için işlem yap
        for row in reader:
            word = row[0]  # Kelimeyi al

            # Kelimenin somut veya soyut olup olmadığını kontrol et ve ilgili değeri ekleyin
            if word in concrete_words:
                row.append('Somut')
            elif word in abstract_words:
                row.append('Soyut')
            else:
                row.append('')  # Eşleşme yoksa boş bırakın

            writer.writerow(row)  # Satırı yaz

print("Yeni CSV dosyası oluşturuldu.")


Yeni CSV dosyası oluşturuldu.


In [87]:
import csv
import string

# Somut kelimelerin olduğu dosyayı oku ve kelimeleri bir küme olarak yükle
concrete_words = set()
with open('400-700.txt', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter=' ')
    for row in reader:
        word = row[0].lower().strip(string.punctuation)
        concrete_words.add(word)

# Soyut kelimelerin olduğu dosyayı oku ve kelimeleri bir küme olarak yükle
abstract_words = set()
with open('100-400.txt', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter=' ')
    for row in reader:
        word = row[0].lower().strip(string.punctuation)
        abstract_words.add(word)

# Kendi oluşturduğunuz CSV dosyasını oku ve her bir kelime için soyut/somut bilgisini kontrol et
with open('ozellikler2.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    
    # Yeni CSV dosyasını yazmak için çıktı dosyasını aç
    with open('sonuclar.csv', 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        header = next(reader)  # Başlık satırını geç
        header.append('Abstract/Concrete')  # Yeni sütun başlığını ekle
        writer.writerow(header)  # Başlık satırını yaz

        # Her bir kelime için işlem yap
        for row in reader:
            word = row[0].lower().strip(string.punctuation)  # Kelimeyi al, küçük harfe dönüştür ve noktalama işaretlerini kaldır

            # Kelimenin somut veya soyut olup olmadığını kontrol et ve ilgili değeri ekleyin
            if word in concrete_words:
                row.append('Concrete')
            elif word in abstract_words:
                row.append('Abstract')
            else:
                row.append('')  # Eşleşme yoksa boş bırakın

            writer.writerow(row)  # Satırı yaz

print("Yeni CSV dosyası oluşturuldu.")


Yeni CSV dosyası oluşturuldu.


In [88]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Sonuçlar dosyasını yükle
data = pd.read_csv('sonuclar.csv')

# Veri öznitelikleri ve hedef değişkeni
X = data.iloc[:, 0].values  # Öznitelik: Kelimeler
y = data.iloc[:, -1].values  # Hedef: Soyut/Somut

# Metin verilerini sayısal özniteliklere dönüştürmek için TF-IDF vektörleştiriciyi kullan
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

# Eğitim ve test veri setlerini oluştur
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression modelini eğit
model = LogisticRegression()
model.fit(X_train, y_train)

# Test veri seti üzerinde tahmin yap
y_pred = model.predict(X_test)

# Sınıflandırma raporunu ekrana yazdır
print(classification_report(y_test, y_pred))


ValueError: Input contains NaN

In [90]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Sonuçlar dosyasını yükle
data = pd.read_csv('sonuclar.csv')

# Eksik değerleri kontrol et
print(data.isnull().sum())

# Eksik değerleri doldur
data = data.dropna()

# Veri öznitelikleri ve hedef değişkeni
X = data.iloc[:, 0].values  # Öznitelik: Kelimeler
y = data.iloc[:, -1].values  # Hedef: Soyut/Somut

# Metin verilerini sayısal özniteliklere dönüştürmek için TF-IDF vektörleştiriciyi kullan
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

# Eğitim ve test veri setlerini oluştur
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression modelini eğit
model = LogisticRegression()
model.fit(X_train, y_train)

# Test veri seti üzerinde tahmin yap
y_pred = model.predict(X_test)

# Sınıflandırma raporunu ekrana yazdır
print(classification_report(y_test, y_pred))


Word                    0
POS                     0
POS Before 1            1
POS Before 2            2
POS After 1             1
POS After 2             2
Positive                0
Negative                0
Abstract/Concrete    3241
dtype: int64
              precision    recall  f1-score   support

       Somut       1.00      0.44      0.61       124
       Soyut       0.92      1.00      0.96       748

    accuracy                           0.92       872
   macro avg       0.96      0.72      0.79       872
weighted avg       0.93      0.92      0.91       872



In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Sonuçlar dosyasını yükle
data = pd.read_csv('sonuclar.csv')

# Eksik değerleri kontrol et
print(data.isnull().sum())

# Eksik değerleri doldur
data = data.dropna()

# Veri öznitelikleri ve hedef değişkeni
X = data.iloc[:, 0].values  # Öznitelik: Kelimeler
y = data.iloc[:, -1].values  # Hedef: Soyut/Somut

# Metin verilerini sayısal özniteliklere dönüştürmek için TF-IDF vektörleştiriciyi kullan
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

# Eğitim ve test veri setlerini oluştur
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression modelini eğit
model = LogisticRegression()
model.fit(X_train, y_train)

# Test veri seti üzerinde tahmin yap
y_pred = model.predict(X_test)

# Sınıflandırma raporunu ekrana yazdır
print(classification_report(y_test, y_pred))

# Yeni satırı tahminlerle birlikte sonuç dosyasına ekle
data['Prediction'] = model.predict(X)
data.to_csv('sonuclar_tahminler.csv', index=False)


Word                    0
POS                     0
POS Before 1            1
POS Before 2            2
POS After 1             1
POS After 2             2
Positive                0
Negative                0
Abstract/Concrete    3241
dtype: int64
              precision    recall  f1-score   support

    Abstract       0.92      1.00      0.96       748
    Concrete       1.00      0.44      0.61       124

    accuracy                           0.92       872
   macro avg       0.96      0.72      0.79       872
weighted avg       0.93      0.92      0.91       872



In [27]:
import pandas as pd

# Tahmin sonuçlarının bulunduğu dosyayı yükle
predictions = pd.read_csv('sonuclar_tahminler.csv')

# Soyut olan ve soyut tahmin edilen kelimeleri belirle
soyut_kelime_tahminleri = predictions[predictions['Abstract/Concrete'] == 'Abstract']
soyut_kelime_dogru_tahminler = soyut_kelime_tahminleri[soyut_kelime_tahminleri['Prediction'] == 'Abstract']

# Metin dosyasını oku ve cümleleri bir liste olarak al
with open('Cat.txt', 'r', encoding='utf-8') as file:
    metin = file.read()
    cumleler = metin.split('.')
    total_cumle_sayisi = len(cumleler)

# Soyut kelime doğru tahmin edilen cümle sayısını hesapla
soyut_dogru_tahmin_cumle_sayisi = 0
for cumle in cumleler:
    cumle_kelimeleri = cumle.strip().split()
    if any(kelime in soyut_kelime_dogru_tahminler['Word'].values for kelime in cumle_kelimeleri):
        soyut_dogru_tahmin_cumle_sayisi += 1

# Yüzdeyi hesapla
yuzde = (soyut_dogru_tahmin_cumle_sayisi / total_cumle_sayisi) * 100

# Sonucu ekrana yazdır
print(f"Doğru tahmin edilen soyut kelimelerin yüzdesi: {yuzde}%")

Doğru tahmin edilen soyut kelimelerin yüzdesi: 98.7146529562982%
